<a href="https://colab.research.google.com/github/aishwarya-kumar/skillrec_for_gigworkers/blob/main/Label_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from collections import Counter
import json

In [8]:
csv_list = ['worker_profiles_DA', 'worker_profiles_DS']

In [9]:
dataframes = []
for file_name in csv_list:
    file_path = "/content/"+f"{file_name}.csv"
    df = pd.read_csv(file_path)
    print(f"Shape of {file_name}: {df.shape}")
    dataframes.append(df)

Shape of worker_profiles_DA: (110, 11)
Shape of worker_profiles_DS: (200, 11)


In [10]:
worker_profiles = pd.concat(dataframes, ignore_index=True)
worker_profiles = worker_profiles.loc[:, ~worker_profiles.columns.str.contains('^Unnamed')]
worker_profiles.shape

(310, 10)

In [11]:
print("Missing values:\n", worker_profiles.isnull().sum())

Missing values:
 title                0
pp                   0
summary              0
total_jobs           0
total_hours          0
locality             0
country             37
perc_job_success     2
projects             0
skills_list          0
dtype: int64


In [12]:
worker_profiles['country'] = worker_profiles['country'].fillna('')

In [13]:
def convert_price(pp):
    return float(pp.replace('$', '').replace('/hr', ''))

worker_profiles['hourly_rate'] = worker_profiles['pp'].apply(convert_price)
worker_profiles.head(2)

,title,pp,summary,total_jobs,total_hours,locality,country,perc_job_success,projects,skills_list,hourly_rate
0,Data Analyst,$15.00/hr,Hello! I am a highly skilled data analyst with...,174,277,Tunis,Tunisia,86.0,"['Pricing exercise', 'Medical Artifical Intell...","['SAS', 'Machine Learning', 'Microsoft Excel',...",15.0
1,Data Analyst and Management Consultant,$8.00/hr,I have great command over data analytics softw...,800,"1,273",Lahore,Pakistan,93.0,"['Back-End Development Back-nd Development', '...","['Data Analysis', 'R', 'Python', 'IBM SPSS', '...",8.0


In [23]:
def generate_prompt(profile):
    title = profile["title"]
    pay_per_hour = profile["hourly_rate"]
    skills = profile["skills_list"]
    total_jobs = profile["total_jobs"]
    perc_job_success = profile["perc_job_success"]
    projects = profile["projects"]
    if len(projects.split()) > 50:  # Truncate summary to 50 words
      projects = " ".join(projects.split()[:50]) + "..."

    profile["projects"] = projects

    # Create a prompt asking for skill recommendations
    prompt = (f"Profile: Job Title: {title}, Skills: {skills}, "
              f"Projects: {projects}, "
              f"Earnings: {pay_per_hour}, Total Jobs: {total_jobs}, "
              f"Success Rate: {perc_job_success}%.\n"
              f"You are a career counsellor for gig workers. You need to understand the profile and backgrounds of the workers and suggest possible ways to upskill themsleves."
              f"What skills should this worker develop next? HOw can they do better with new skills?")
    return prompt

In [24]:
worker_profiles["prompt"] = worker_profiles.apply(generate_prompt, axis=1)
worker_profiles.head(2)

,title,pp,summary,total_jobs,total_hours,locality,country,perc_job_success,projects,skills_list,hourly_rate,prompt
0,Data Analyst,$15.00/hr,Hello! I am a highly skilled data analyst with...,174,277,Tunis,Tunisia,86.0,"['Pricing exercise', 'Medical Artifical Intell...","['SAS', 'Machine Learning', 'Microsoft Excel',...",15.0,"Profile: Job Title: Data Analyst, Skills: ['SA..."
1,Data Analyst and Management Consultant,$8.00/hr,I have great command over data analytics softw...,800,"1,273",Lahore,Pakistan,93.0,"['Back-End Development Back-nd Development', '...","['Data Analysis', 'R', 'Python', 'IBM SPSS', '...",8.0,Profile: Job Title: Data Analyst and Managemen...


In [25]:
prompts1 = worker_profiles[["prompt"]]

In [26]:
prompts1

,prompt
0,"Profile: Job Title: Data Analyst, Skills: ['SA..."
1,Profile: Job Title: Data Analyst and Managemen...
2,Profile: Job Title: Excel Google Sheets Expert...
3,Profile: Job Title: Excel Data Analyst Expert ...
4,Profile: Job Title: Data Analyst | SPSS | STAT...
...,...
305,Profile: Job Title: Machine Learning & AI Spec...
306,"Profile: Job Title: Data Scientist, Skills: ['..."
307,Profile: Job Title: Data Science & AI Expert -...
308,Profile: Job Title: Data Scientist | Data Engi...


LLM

In [27]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [19]:
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [28]:
def generate_recommendations(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    outputs = model.generate(inputs["input_ids"], max_length=50)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [ ]:
prompts1["response"] = prompts1["prompt"].apply(generate_recommendations)

In [ ]:
prompts1["response"]